# Filter May - Oct 2000 NYT corpus for election-related paragraphs

In [7]:
import os
import xml.etree.ElementTree as ET

def find_election_paragraphs(directory):
    election_paragraphs = open("election_paragraphs.txt", "w+")
    for root, dirs, files in os.walk(directory):
        for file in files:
            f = open(os.path.join(root, file), "r")
            parse_xml(f, election_paragraphs)

            
def parse_xml(xml, election_paragraphs):
    tree = ET.parse(xml)
    root = tree.getroot()
    for person in root.findall('.//person'):
        name = person.text
        if contains_election_words(name):
            for block in root.findall('.//block'):
                if block.attrib['class'] == 'full_text':
                    for paragraph in block.findall('.//p'):
                        if contains_election_words(paragraph.text):
                            election_paragraphs.write(paragraph.text + '\n')
            break

    
def contains_election_words(text):
    return "Bush" in text or "Gore" in text
    
            

In [14]:
import pandas as pd

paragraphs_df = pd.read_csv("election_paragraphs.txt", delimiter = "\n", header = None)
paragraphs_df

,0
0,Following are excerpts from remarks prepared f...
1,"From what we can tell of his foreign policy, G..."
2,Governor Bush dangerously fixates on the cold ...
3,"Stuck in a cold war mindset, Governor Bush con..."
4,"Just this past week, Governor Bush used his br..."
...,...
1608,''We have a national consensus about the impor...
1609,Mr. Bush said the proposal he made today would...
1610,The plan would set aside half of that money fo...
1611,"Today's speech followed, by only two days, a m..."


# Apply LDA to the paragraph set to identify candidate topics

## Clean document set